# Demo for XYZ Spaces for Python

This notebook shows a demo of xyzspaces.

In [ ]:
# This is only to show what to integrate better into Jupyter in the future
# (not covered further in this notebook).

from IPython.display import IFrame

url = "https://xyz.here.com/viewer/"
project_id = "5c54716d-f900-4b89-80ac-b21518e94b30"
IFrame(src=f"{url}?project_id={project_id}", width="100%", height=400)

In [ ]:
import os

import requests
from sidecar import Sidecar
from ipyleaflet import Map, GeoJSON, FullScreenControl

from xyzspaces.apis import HubApi
from xyzspaces.datasets import get_countries_data

In [ ]:
xyz_token = os.environ["XYZ_TOKEN"]
api = HubApi(credentials=xyz_token)

# create a temporary space
res = api.post_space(
    data={"title": "My Demo Space",
          "description": "Demo"})
space_id = res["id"]

In [ ]:
# create map widget in sidebar (JupyterLab needed)
m = Map(zoom=2)
with Sidecar(title='XYZ'):
    display(m)

In [ ]:
# load https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json
# with minor clean-ups
gj = get_countries_data()

In [ ]:
# add world GeoJSON to XYZ space (returns the added GeoJSON)
world = api.put_space_features(
    space_id=space_id, 
    data=gj)

In [ ]:
# add entire GeoJSON feature from XYZ to the map
m += GeoJSON(data=world)

In [ ]:
# add "full screen" button to map
m += FullScreenControl()

In [ ]:
# get one feature (France) from XYZ space
fra = api.get_space_feature(
    space_id=space_id,
    feature_id="FRA"
)

In [ ]:
# add the feature to the map
style = dict(
    color="red",
    opacity=1,
    weight=1.9, 
    fillOpacity=0.5
)
m += GeoJSON(data=fra, style=style)

In [ ]:
# show only countries on southern hemisphere 
bbox = [-180, -90, 180, 0]
kwargs = dict(space_id=space_id, bbox=bbox)
hemi = api.get_space_bbox(**kwargs)
m += GeoJSON(data=hemi, style=style)

In [ ]:
# show the same but with clipped borders 
kwargs["clip"] = True
hemi = api.get_space_bbox(**kwargs)
style["color"] = "yellow"
m += GeoJSON(data=hemi, style=style)

In [ ]:
# Spatial search: Given latitude and longitude search intersecting geometry.

spatial_search = api.get_space_spatial(space_id=space_id, lat=19.1526, lon=72.8557)
spatial_search
style["color"] = "orange" 
m += GeoJSON(data=spatial_search, style=style)
m

In [ ]:
# clean-up and delete temporary space
api.delete_space(space_id=space_id)